In [1]:
! pip install pyautogen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


## Write `.py` Script

Write a python script locally assuming `code` is provided.

In [6]:
from pathlib import Path

from autogen.coding import CodeBlock, LocalCommandLineCodeExecutor

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

executor = LocalCommandLineCodeExecutor(work_dir=work_dir)
user_name = input("What is your name? ")
response = executor.execute_code_blocks(
        code_blocks=[
            CodeBlock(language="python", code=f"print('Hello {user_name}!')"),
        ]
    )

print(response.output)

What is your name? Tom
Hello Tom!



## Write and Execute `.py` Script

Write a python script using natural language and execute it.

In [7]:
from autogen import ConversableAgent

In [8]:
# Create an agent with code executor configuration.
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
)

In [9]:
message_with_code_block = """This is a message with code block.
The code block is below:
```python
import numpy as np
import matplotlib.pyplot as plt
x = np.random.randint(0, 100, 100)
y = np.random.randint(0, 100, 100)
plt.scatter(x, y)
plt.savefig('scatter.png')
print('Scatter plot saved to scatter.png')
```
This is the end of the message.
"""

# Generate a reply for the given code.
reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
print(reply)

Provide feedback to the sender. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 0 (execution succeeded)
Code output: Scatter plot saved to scatter.png



## Write and Execute `.py` Script from Human Question

In [11]:
! pip install openai

In [12]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [13]:
from openai import OpenAI


class ChatBot:
    def __init__(self):
        self.client = OpenAI(api_key=OPENAI_API_KEY)
        self.history = [{"role": "system", "content": "You are a helpful assistant."}]

    def generate_response(self, prompt: str) -> str:
        self.history.append({"role": "user", "content": prompt})

        completion = self.client.chat.completions.create(
            model="gpt-3.5-turbo", # NOTE: feel free to change it to gpt-4, or gpt-4o
            messages=self.history
        )

        response = completion.choices[0].message.content
        self.history.append({"role": "assistant", "content": response})

        return response

    def get_history(self) -> list:
        return self.history

In [14]:
# message_with_code_block = """This is a message with code block.
# The code block is below:
# ```python
# import numpy as np
# import matplotlib.pyplot as plt
# x = np.random.randint(0, 100, 100)
# y = np.random.randint(0, 100, 100)
# plt.scatter(x, y)
# plt.savefig('scatter.png')
# print('Scatter plot saved to scatter.png')
# ```
# This is the end of the message.
# """

In [18]:
# Initiate bot
bot = ChatBot()
bot.generate_response(input("Ask me a python function: "))
bot.get_history()[-1]["content"]

Ask me a python function: write a hello world python function for me


'Sure, here is a simple "Hello, World!" function in Python:\n\n```python\ndef hello_world():\n    print("Hello, World!")\n\nhello_world()\n```'

In [21]:
# Initiate bot
bot = ChatBot()
bot.generate_response(input("Ask me a python function: "))
message_with_code_block = bot.get_history()[-1]["content"]
print(message_with_code_block)

# Generate a reply for the given code.
reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
print(reply)

Ask me a python function: I want a python function that computes the nth fibonacci number. Let's compute for the 3rd number. Print it.
Sure! Here's a Python function to calculate the nth Fibonacci number:

```python
def fibonacci(n):
    if n <= 0:
        return "Input should be a positive integer"
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        fib_list = [0, 1]
        for i in range(2, n):
            fib_list.append(fib_list[i-1] + fib_list[i-2])
        return fib_list[-1]

# Compute the 3rd Fibonacci number
result = fibonacci(3)
print(result)
```

When you run this code, it will print the 3rd Fibonacci number, which is 1.
Provide feedback to the sender. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 0 (execution succeeded)
Code output: 1

